In [13]:
import numpy as np
import matplotlib.pyplot as plt

# 卡尔曼滤波器（Kalman Filter，KF）算法及仿真

## 1. 贝叶斯滤波器

贝叶斯滤波器的算法如下所示：

----
1：**Algorithm Bayes_filter**$(bel(x_{t-1}), u_t, z_t)$**:**  
2：&emsp;&emsp;for all $x_t$ do  
3：&emsp;&emsp;&emsp;&emsp;$\overline {bel}(x_t) = \int p(x_t | u_t, x_{t-1})bel(x_{t-1})dx_{t-1}$  
4：&emsp;&emsp;&emsp;&emsp;$bel(x_t)=\eta p ( z_t | x_t) \overline {bel}(x_t)$  
5：&emsp;&emsp;end for  
6：&emsp;&emsp;**return** $bel(x_t)$  

----

**贝叶斯滤波**的生成模型也被称为**隐马尔科夫模型（Hidden Markov Model，HMM）**或**动态贝叶斯网络（Dynamic Bayes Network， DBN）**。

对于离散有限的状态空间，可以用所有可能状态的概率值来表示置信度。但是如果状态空间是无限或连续，则置信度难以表示。

通常有两种置信度的表示方法：
1. 用多元正态分布来表示置信度。对应的滤波算法被称统称为**高斯滤波（Gaussian Filter）**。  
2. 用粒子集来表示置信度。对应的滤波算法被称为**粒子滤波（Particle Filter）**。  

## 2. 卡尔曼滤波

KF算法用**矩**来定义置信度，可以对连续状态的置信度进行计算，但是不适用于离散状态空间和混合状态空间。

----
1：**Algorithm Kalman_filter**$( \mu, \Sigma_{t-1}, u_t, z_t )$**:**  
2：&emsp;&emsp;$\overline \mu_t = A_t \mu_{t-1} + B_t u_t$  
3：&emsp;&emsp;$\overline \Sigma_t = A_t \Sigma_{t-1}A_t^T + R_t$  
4：&emsp;&emsp;$K_t = \overline \Sigma_t C_t^T ( C_t \overline \Sigma_t C_t^T + Q_t)^{-1}$  
5：&emsp;&emsp;$\mu_t = \overline \mu_t + K_t (z_t - C_t \overline \mu_t)$  
6：&emsp;&emsp;$\Sigma_t = (I - K_t C_t) \overline \Sigma_t$  
7：&emsp;&emsp;**return** ${\mu_t, \Sigma_t}$  

----


## 3. Kalman Filter仿真


### 3.1 仿真模型

#### 3.1.1 问题描述

假设有一个沿直线运动的机器人，每隔一个时间间隔$\vartriangle t$，根据运动$u_t$对自己当前的状态进行预测（Predict），然后利用传感器获得的测量$z_t$对自己的状态进行更新（Update）。

定义机器人的状态$x_t = \begin{bmatrix}p_t \\ v_t \end{bmatrix}$，其中$p_t$表示机器人的位置，$v_t$表示机器人的速度。

#### 3.1.2 运动模型

假设机器人可以控制其加速度$a$，则运动迁移方程可以写为：
$$\begin{align}
p_t &= p_{t-1} + v_{t-1} \triangle t + \frac 12 a \triangle t^2  \\
v_t &= v_{t-1} + a \triangle t
\end{align}$$

把该方程改写为线性方程，可得：
$$\begin{align}
x_t &= \begin{bmatrix} p_t \\ v_t \end{bmatrix} = 
\begin{bmatrix} 1 & \triangle t \\ 0 & 1 \end{bmatrix} \begin{bmatrix} p_{t-1} \\ v_{t-1} \end{bmatrix} +
\begin{bmatrix} \frac 12  \triangle t^2 \\ \triangle t \end{bmatrix}  a + \varepsilon_t
\\
 &= A_t x_{t-1} + B_t u_t + \varepsilon_t
\end{align}
$$

其中$\varepsilon_t$是控制带来的噪声，假设加速度$a$包含有一个$\sim N(\mu , \sigma_a^2)$的一个噪声。因此该噪声对机器人状态$x_t$带来的噪声可表示为：$ noise_u \sim N(\mu , R_t)$，其中$\mu = (0, 0)^T$，$R_t = \begin{bmatrix}\frac 12 \triangle t_2  & 0 \\ 0 & \triangle t \end{bmatrix} \sigma_u^2$。

#### 3.1.3 测量模型

贝叶斯滤波的测量模型并不是使用后验概率$p(x \, \vert \, z)$，而是用似然（likelihood）来进行计算。
$$z_t = C_t x_t + \delta_t $$

假设该仿真模型中的机器人可以获得对当前位置的一个测量量$z_t$，因此可以得到测量模型为：
$$\begin{align}
z_t &= p_t + \delta_t \\
   &= \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} p_t \\ v_t \end{bmatrix} + \delta_t \\
   &= C_t x_t + \delta_t   
\end{align}$$

其中$\delta_t$是测量的噪声，该噪声遵从一个均值为0的高斯分布$\delta_t \sim N( \mu , \sigma_z^2)$。

## 3.2 仿真

### 3.2.1 运动采样函数

在仿真过程中，需要